In [ ]:
import pandas as pd #
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "./data/Landslide_dataSet1.csv"
device = "102"
save_path = f"./data/dev{device}_prepared.csv"
#save_path = f"data/devAll2_prepared.csv"

df = pd.read_csv(csv_path)

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

def clean_and_format_timestamp(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('(ICT)', '').strip()
    try:
        dt = pd.to_datetime(val)
        return dt.isoformat()
    except Exception:
        return None  

df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])

df = df.drop(columns=['unixt', 'gateway time', 'old time', 'risk_level', 'confidence'], errors='ignore')
df = df[df['devID'] != 218499049.0]
df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
df = df[df['lng'] > 15.0]
df = df[df['soil'] != 0.0]
df = df[df['rain'] != -1.0]

df = df[df['devID'] == int(device)]
df

,timestamp,devID,soil_mean,rain_mean,temp_mean,humi_mean,geo_mean,geo_std,label
0,2025-05-02 11:40:00,102,22.200001,0.0,34.717920,62.513730,7.895913,5.625963,NaN
1,2025-05-02 11:50:00,102,22.203572,0.0,34.847077,61.514394,9.278016,7.178730,NaN
2,2025-05-02 12:00:00,102,22.200001,0.0,35.709997,61.343821,9.139770,6.965634,NaN
3,2025-05-02 12:10:00,102,22.203704,0.0,34.937530,62.299312,7.868829,6.421354,NaN
4,2025-05-02 12:20:00,102,22.125000,0.0,34.723956,63.180795,8.939695,5.708089,NaN
...,...,...,...,...,...,...,...,...,...
3331,2025-06-29 19:40:00,102,25.600000,0.0,29.510186,80.122087,5.407425,4.375503,NaN
3332,2025-06-29 19:50:00,102,25.600000,0.0,29.486474,80.045778,5.613575,3.926898,NaN
3333,2025-06-29 20:00:00,102,25.600000,0.0,29.724941,78.549214,8.002298,5.757017,NaN
3334,2025-06-29 20:10:00,102,25.600000,0.0,29.742026,78.720270,8.012238,5.920143,NaN


In [2]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
count,375609,375609.000000,375609.000000,375609.000000,375609.000000,375609.000000,375609.000000,375609.000000,375609.000000
mean,2025-06-11 07:41:09.393899008,101.502898,43.992966,0.165806,29.900678,74.567455,8.239718,8.637978,99.901333
min,2025-05-02 11:42:00.384119,101.000000,20.200001,0.000000,24.238958,43.418247,0.000000,8.637330,99.166664
25%,2025-05-30 13:50:33.973680896,101.000000,28.500000,0.000000,27.397957,68.698631,3.001680,8.637944,99.898926
50%,2025-06-13 21:13:00.729999872,102.000000,46.799999,0.000000,28.586250,78.603645,4.549040,8.637974,99.898949
75%,2025-06-27 15:10:47.049999872,102.000000,56.000000,0.000000,32.695885,80.863892,13.630129,8.638015,99.898972
max,2025-07-22 19:39:55.017000,102.000000,66.599998,41.630600,39.331657,88.510567,32.062466,8.638361,995.549988
std,NaN,0.499992,13.708053,1.579886,3.189388,8.844493,6.654768,0.000038,1.461406


In [3]:
df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
0,2025-06-27 18:59:34.950000,101.0,41.500000,0.0,28.444725,79.391396,12.150635,8.638015,99.898972
1,2025-05-02 11:42:00.384119,102.0,22.200001,0.0,34.549099,61.175941,3.323395,8.637964,99.898933
2,2025-05-02 11:42:06.635276,101.0,28.799999,0.0,34.290073,62.429081,14.532875,8.637999,99.899010
3,2025-05-02 11:42:11.034615,102.0,22.200001,0.0,34.506374,61.542152,14.766250,8.637963,99.898933
4,2025-05-02 11:42:18.056572,101.0,28.799999,0.0,34.266041,64.082787,14.673310,8.637998,99.899010
...,...,...,...,...,...,...,...,...,...
383609,2025-07-22 19:39:28.596000,101.0,25.600000,0.0,28.682386,74.146103,7.982772,8.638018,99.898956
383610,2025-07-22 19:39:35.152000,102.0,23.299999,0.0,28.653008,73.873352,5.533101,8.637944,99.898926
383611,2025-07-22 19:39:41.789000,101.0,25.600000,0.0,28.674376,74.270081,0.931764,8.638016,99.898956
383612,2025-07-22 19:39:48.552000,102.0,23.299999,0.0,28.631645,73.905777,6.762322,8.637944,99.898926


In [22]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
df = df.set_index('timestamp')
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek
df = df.reset_index() 

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3336 entries, 0 to 3335
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   timestamp    3336 non-null   datetime64[ns]
 1   devID        3336 non-null   int64         
 2   soil_mean    3336 non-null   float64       
 3   rain_mean    3336 non-null   float64       
 4   temp_mean    3336 non-null   float64       
 5   humi_mean    3336 non-null   float64       
 6   geo_mean     3336 non-null   float64       
 7   geo_std      3336 non-null   float64       
 8   label        21 non-null     object        
 9   hour         3336 non-null   int32         
 10  day_of_week  3336 non-null   int32         
dtypes: datetime64[ns](1), float64(6), int32(2), int64(1), object(1)
memory usage: 260.8+ KB


In [19]:
nan_counts = df.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
label    3158
dtype: int64


In [7]:
df = df.dropna(subset=['lng']).reset_index(drop=True)

In [8]:
nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [9]:
df = df.dropna()

nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("df102")
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

df102
📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [24]:

df.to_csv(f"{save_path}", index=False)

print("✅ บันทึกเรียบร้อยแล้ว")

✅ บันทึกเรียบร้อยแล้ว
